# Joint multitask

The point of this notebook is not to do anything useful, but to show what is possible with relatively low effort using the *transformer_heads* library. In this example, we will train four heads on a transformer model while using qlora to finetune the transformer block weights. The first head will be hooked at layer 9 (-4) and predict the sentiment of imdb reviews (text classification). The second head will be hooked at the last layer (-1 or 12) and does causal language modelling on imdb reviews. The third head will be hooked at layer 6 (-7) and will learn to count the number of occurences of each letter of the alphabet occuring in imdb reviews (Text-level regression). The final head will be hooked at layer 4 (-9) and will predict how many tokens will follow before the review ends for each token in imdb reviews (Token-level regression). The final head will also be a small mlp instead of a linear head.

All heads and the qlora parameters will be trained jointly (multi-task learning).

In [1]:
from transformer_heads import create_headed_qlora, load_lora_with_heads
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    MistralForCausalLM,
    Trainer,
    BitsAndBytesConfig,
    TrainingArguments,
    GPT2Model,
    GPT2LMHeadModel,
)
from transformer_heads.util.helpers import DataCollatorWithPadding, get_model_params
from peft import LoraConfig
from transformer_heads.config import HeadConfig
from transformer_heads.util.model import print_trainable_parameters
from transformer_heads.util.evaluate import (
    evaluate_head_wise,
)
import torch
import pandas as pd

In [2]:
# GPT2 is the fastest and requires fewest memory. However, this works just the same with any Llama or Mistral model. Just change model_path to its huggingface path.
model_path = "gpt2"
train_epochs = 1
eval_epochs = 1
logging_steps = 100
train_batch_size = 4
eval_batch_size = 4

In [3]:
# Parameters
model_path = "gpt2"
train_epochs = 1
eval_epochs = 1
logging_steps = 100

In [4]:
model_params = get_model_params(model_path)
model_class = model_params["model_class"]
hidden_size = model_params["hidden_size"]
vocab_size = model_params["vocab_size"]
print(model_params)

{'vocab_size': 50257, 'n_positions': 1024, 'n_embd': 768, 'n_layer': 12, 'n_head': 12, 'n_inner': None, 'activation_function': 'gelu_new', 'resid_pdrop': 0.1, 'embd_pdrop': 0.1, 'attn_pdrop': 0.1, 'layer_norm_epsilon': 1e-05, 'initializer_range': 0.02, 'summary_type': 'cls_index', 'summary_use_proj': True, 'summary_activation': None, 'summary_first_dropout': 0.1, 'summary_proj_to_labels': True, 'scale_attn_weights': True, 'use_cache': True, 'scale_attn_by_inverse_layer_idx': False, 'reorder_and_upcast_attn': False, 'bos_token_id': 50256, 'eos_token_id': 50256, 'return_dict': True, 'output_hidden_states': False, 'output_attentions': False, 'torchscript': False, 'torch_dtype': None, 'use_bfloat16': False, 'tf_legacy_loss': False, 'pruned_heads': {}, 'tie_word_embeddings': True, 'chunk_size_feed_forward': 0, 'is_encoder_decoder': False, 'is_decoder': False, 'cross_attention_hidden_size': None, 'add_cross_attention': False, 'tie_encoder_decoder': False, 'max_length': 20, 'min_length': 0, '

Define the various different heads. Given the differences in loss functions and magnitudes of label data in the dataset, it is important to weigh the losses of each head so that training is given similar importance for all of them.

In [5]:
head_configs = [
    HeadConfig(
        name=f"sentiment_head",
        layer_hook=-4,
        in_size=hidden_size,
        output_activation="linear",
        pred_for_sequence=True,
        loss_fct="cross_entropy",
        num_outputs=2,
        loss_weight=2.0,
    ),
    HeadConfig(
        name=f"causal_lm",
        layer_hook=-1,
        in_size=hidden_size,
        output_activation="linear",
        is_causal_lm=True,
        loss_fct="cross_entropy",
        num_outputs=vocab_size,
        is_regression=False,
        output_bias=False,
        loss_weight=1.0,
    ),
    HeadConfig(
        name=f"alphabet_regression",
        layer_hook=-7,
        in_size=hidden_size,
        output_activation="linear",
        is_causal_lm=False,
        pred_for_sequence=True,
        loss_fct="mse",
        num_outputs=26,  # 26 letters in the alphabet
        is_regression=True,
        loss_weight=0.002,
    ),
    HeadConfig(
        name=f"num_tokens_regression",
        layer_hook=-7,
        hidden_size=128,  # MLP hidden size
        num_layers=3,  # 2 hidden layers in MLP
        in_size=hidden_size,
        output_activation="linear",
        is_causal_lm=False,
        pred_for_sequence=False,
        loss_fct="mse",
        num_outputs=1,
        is_regression=True,
        loss_weight=0.0002,
    ),
    HeadConfig(
        name=f"lm_head",  # Let's also keep the original lm head for comparison
        layer_hook=-1,
        in_size=hidden_size,
        output_activation="linear",
        is_causal_lm=True,
        pred_for_sequence=False,
        loss_fct="cross_entropy",
        num_outputs=vocab_size,
        is_regression=False,
        trainable=False,  # Keep it in it's pretrained state
    ),
]

In [6]:
dd = load_dataset("imdb")

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token


def processing_function(examples):
    out = tokenizer(examples["text"], padding=False, truncation=True)
    out["sentiment_head"] = examples["label"]
    out["causal_lm"] = out["lm_head"] = out["input_ids"].copy()
    out["num_tokens_regression"] = [
        list(map(float, range(len(ids) - 1, -1, -1))) for ids in out["input_ids"]
    ]
    out["alphabet_regression"] = [
        [
            float(text.count(x) + text.count(x.upper()))
            for x in "abcdefghijklmnopqrstuvwxyz"
        ]
        for text in examples["text"]
    ]
    return out


for split in dd.keys():
    dd[split] = dd[split].filter(function=lambda example: len(example["text"]) > 10)
    dd[split] = dd[split].shuffle()
    dd[split] = dd[split].map(processing_function, batched=True)

dd.set_format(
    type="torch",
    columns=["input_ids", "attention_mask"] + [x.name for x in head_configs],
)
for split in dd.keys():
    dd[split] = dd[split].remove_columns(["text", "label"])

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
dd["test"]

Dataset({
    features: ['input_ids', 'attention_mask', 'sentiment_head', 'causal_lm', 'lm_head', 'num_tokens_regression', 'alphabet_regression'],
    num_rows: 25000
})

Setting *target_modules=None* in the qlora config will make *create_headed_qlora* create LoRA modules for all linear layers in the transformer.

In [9]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_in_8bit=False,
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_compute_dtype=torch.float32,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)
lora_config = LoraConfig(
    r=32,
    lora_alpha=16,
    target_modules=None,
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
)
model = create_headed_qlora(
    base_model_class=model_class,
    model_name=model_path,
    quantization_config=quantization_config,
    lora_config=lora_config,
    head_configs=head_configs,
    fully_trained_heads=True,
    device_map={"": torch.cuda.current_device()},
    gradient_checkpointing=True,
)

Some weights of TransformerWithHeads were not initialized from the model checkpoint at gpt2 and are newly initialized: ['heads.alphabet_regression.lins.0.weight', 'heads.causal_lm.lins.0.weight', 'heads.num_tokens_regression.lins.0.bias', 'heads.num_tokens_regression.lins.0.weight', 'heads.num_tokens_regression.lins.1.bias', 'heads.num_tokens_regression.lins.1.weight', 'heads.num_tokens_regression.lins.2.weight', 'heads.sentiment_head.lins.0.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
print_trainable_parameters(model)

all params: 125425024 || trainable params: 43452544 || trainable%: 34.64423813863492
params by dtype: defaultdict(<class 'int'>, {torch.float32: 82957696, torch.uint8: 42467328})
trainable params by dtype: defaultdict(<class 'int'>, {torch.float32: 43452544})


In [11]:
collator = DataCollatorWithPadding(
    feature_name_to_padding_value={
        "input_ids": tokenizer.pad_token_id,
        "attention_mask": 0,
        "causal_lm": -100,
        "lm_head": -100,
        "num_tokens_regression": 0,
    }
)

In [12]:
print(
    evaluate_head_wise(
        model, dd["test"], collator, epochs=eval_epochs, batch_size=eval_batch_size
    )
)


Evaluating:   0%|          | 0/6250 [00:00<?, ?it/s]


Evaluating:   0%|          | 1/6250 [00:00<1:32:19,  1.13it/s]


Evaluating:   0%|          | 2/6250 [00:01<52:02,  2.00it/s]  


Evaluating:   0%|          | 3/6250 [00:01<33:49,  3.08it/s]


Evaluating:   0%|          | 4/6250 [00:01<30:49,  3.38it/s]

...


Evaluating: 100%|█████████▉| 6248/6250 [31:50<00:00,  3.75it/s]


Evaluating: 100%|█████████▉| 6249/6250 [31:51<00:00,  4.05it/s]


Evaluating: 100%|██████████| 6250/6250 [31:51<00:00,  4.08it/s]


Evaluating: 100%|██████████| 6250/6250 [31:51<00:00,  3.27it/s]

(51.503634765014645, {'sentiment_head': 5.004700186492335, 'causal_lm': 22.38208692779541, 'alphabet_regression': 4078.0613999365232, 'num_tokens_regression': 36263.337341796876, 'lm_head': 3.703357028274536})


In [13]:
args = TrainingArguments(
    output_dir="imdb_linear_probe",
    learning_rate=0.0002,
    num_train_epochs=train_epochs,  # To speed things up set to 0.1, set to 1 for better performance
    logging_steps=logging_steps,
    do_eval=False,
    remove_unused_columns=False,
    optim="paged_adamw_32bit",
    gradient_checkpointing=True,
    lr_scheduler_type="constant",
    ddp_find_unused_parameters=False,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
)
trainer = Trainer(
    model,
    args=args,
    train_dataset=dd["train"],
    data_collator=collator,
)
trainer.train()

wandb: Currently logged in as: ykeller (chm-hci). Use `wandb login --relogin` to force relogin


wandb: wandb version 0.16.4 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


wandb: Tracking run with wandb version 0.16.3


wandb: Run data is saved locally in /raven/u/ykeller/transformer_heads/wandb/run-20240323_222852-2unafm5b
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run frosty-salad-210


...

Checkpoint destination directory imdb_linear_probe/checkpoint-5500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Checkpoint destination directory imdb_linear_probe/checkpoint-6000 already exists and is non-empty.Saving will proceed but saved results may be invalid.


/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=6250, training_loss=13.10343646484375, metrics={'train_runtime': 1526.4757, 'train_samples_per_second': 16.378, 'train_steps_per_second': 4.094, 'total_flos': 1.0162662301258752e+16, 'train_loss': 13.10343646484375, 'epoch': 1.0})

In [14]:
evals = evaluate_head_wise(
    model, dd["test"], collator, epochs=eval_epochs, batch_size=eval_batch_size
)
print(evals)


Evaluating:   0%|          | 0/6250 [00:00<?, ?it/s]

/u/ykeller/conda-envs/sh_finetuning/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



Evaluating:   0%|          | 1/6250 [00:00<22:58,  4.53it/s]


Evaluating:   0%|          | 2/6250 [00:00<23:24,  4.45it/s]


Evaluating:   0%|          | 3/6250 [00:00<18:29,  5.63it/s]

...


Evaluating: 100%|█████████▉| 6248/6250 [32:02<00:00,  3.71it/s]


Evaluating: 100%|█████████▉| 6249/6250 [32:02<00:00,  4.00it/s]


Evaluating: 100%|██████████| 6250/6250 [32:03<00:00,  4.06it/s]


Evaluating: 100%|██████████| 6250/6250 [32:03<00:00,  3.25it/s]

(11.534467044143677, {'sentiment_head': 0.32214655859653885, 'causal_lm': 4.127306442260743, 'alphabet_regression': 100.85618892456054, 'num_tokens_regression': 13946.417831679688, 'lm_head': 3.771871594276428})


# Saving and loading
Now let's how to save a complicated mulit-headed model to then load it again for inference. Saving is super easy. Just call save_pretrained and all trained parameters will be saved correctly. The saving will also work correctly for checkpoints created during training.

In [15]:
model.save_pretrained("qlora_multitask_imdb")
del model

While loading the model, we need to make sure to correctly attach and initialize all the heads, so that won't easily work with the huggingface api. Instead, *transformer_heads* provides the *load_lora_with_heads* function. Note that giving a quantization config is optional here. We could also give a different quantization config or none at all.

In [16]:
model = load_lora_with_heads(
    model_class,
    "qlora_multitask_imdb",
    quantization_config,
    device_map={"": torch.cuda.current_device()},
)

Let's now find out if the loaded model behaves the same as the saved model:

In [17]:
new_evals = evaluate_head_wise(
    model, dd["test"], collator, epochs=eval_epochs, batch_size=eval_batch_size
)
print(new_evals)


Evaluating:   0%|          | 0/6250 [00:00<?, ?it/s]


Evaluating:   0%|          | 1/6250 [00:00<23:04,  4.51it/s]


Evaluating:   0%|          | 2/6250 [00:00<23:36,  4.41it/s]


Evaluating:   0%|          | 3/6250 [00:00<18:17,  5.69it/s]


Evaluating:   0%|          | 4/6250 [00:00<21:23,  4.87it/s]

...


Evaluating: 100%|█████████▉| 6248/6250 [31:43<00:00,  3.80it/s]


Evaluating: 100%|█████████▉| 6249/6250 [31:43<00:00,  4.11it/s]


Evaluating: 100%|██████████| 6250/6250 [31:43<00:00,  4.14it/s]


Evaluating: 100%|██████████| 6250/6250 [31:43<00:00,  3.28it/s]

(11.163953929519653, {'sentiment_head': 0.3076160474227845, 'causal_lm': 4.014423089256287, 'alphabet_regression': 105.97276580749512, 'num_tokens_regression': 13420.149343261719, 'lm_head': 3.6383234076690676})
